# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import gmaps
import os
import json

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,0.58,63,96,15.71,TF,1723866860
1,1,bredasdorp,-34.5322,20.0403,7.35,90,0,2.65,ZA,1723866862
2,2,tiksi,71.6872,128.8694,12.97,54,11,2.72,RU,1723866863
3,3,jinchang,38.4953,102.1739,28.06,20,0,4.18,CN,1723866864
4,4,lagunillas,10.1301,-71.2595,26.47,82,42,2.99,VE,1723866866


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
      "Lng", 
      "Lat", 
      geo=True,
      color="City",
      alpha=0.6,
      size="Humidity",
      tiles="OSM"
)

# Display the map
city_map

ImportError: Geographic projection support requires: geoviews.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 26)
                                       & (city_data_df["Max Temp"] > 10) 
                                       & (city_data_df["Cloudiness"] == 0)
                                       & (city_data_df["Wind Speed"] < 5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,ribeira grande,38.5167,-28.7000,22.65,82,0,1.34,PT,1723866870
31,31,chinchilla,-26.7500,150.6333,25.57,44,0,1.34,AU,1723866903
49,49,port alfred,-33.5906,26.8910,18.34,27,0,3.96,ZA,1723866927
58,58,port elizabeth,-33.9180,25.5701,15.73,89,0,4.63,ZA,1723866940
60,60,alice springs,-23.7000,133.8833,23.75,23,0,3.09,AU,1723866943
...,...,...,...,...,...,...,...,...,...,...
558,558,komatipoort,-25.4332,31.9548,15.78,55,0,1.73,ZA,1723867676
564,564,pimentel,-6.8367,-79.9342,16.16,82,0,4.12,PE,1723867684
568,568,baker city,44.7749,-117.8344,18.84,48,0,3.09,US,1723867690
573,573,nizhneangarsk,55.7833,109.5500,18.43,55,0,1.26,RU,1723867698


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,ribeira grande,PT,38.5167,-28.7000,82,
31,chinchilla,AU,-26.7500,150.6333,44,
49,port alfred,ZA,-33.5906,26.8910,27,
58,port elizabeth,ZA,-33.9180,25.5701,89,
60,alice springs,AU,-23.7000,133.8833,23,
...,...,...,...,...,...,...
558,komatipoort,ZA,-25.4332,31.9548,55,
564,pimentel,PE,-6.8367,-79.9342,82,
568,baker city,US,44.7749,-117.8344,48,
573,nizhneangarsk,RU,55.7833,109.5500,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira grande - nearest hotel: Hospedaria JSF
chinchilla - nearest hotel: Chinchilla Downtown Motor Inn
port alfred - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
alice springs - nearest hotel: Aurora Alice Springs
och'amch'ire - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
east london - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
the pas - nearest hotel: Wescana Inn
sao felix do xingu - nearest hotel: Hotel Terraço
sao vicente - nearest hotel: Pousada Vitória
nagqu - nearest hotel: Naqu Hotel
puerto leguizamo - nearest hotel: Hotel Girasoles
aragarcas - nearest hotel: Hotel Portal da Praia
lompoc - nearest hotel: Red Roof Inn Lompoc
fomento - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
port douglas - nearest hotel: Mantra Aqueous Port Douglas
sheridan - nearest hotel: Comfort Inn & Suites
belmonte - nearest hotel: No hotel found
noosa heads - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
7,ribeira grande,PT,38.5167,-28.7000,82,Hospedaria JSF
31,chinchilla,AU,-26.7500,150.6333,44,Chinchilla Downtown Motor Inn
49,port alfred,ZA,-33.5906,26.8910,27,No hotel found
58,port elizabeth,ZA,-33.9180,25.5701,89,Waterford Hotel
60,alice springs,AU,-23.7000,133.8833,23,Aurora Alice Springs
...,...,...,...,...,...,...
558,komatipoort,ZA,-25.4332,31.9548,55,Ecolux Boutique Hotel
564,pimentel,PE,-6.8367,-79.9342,82,Hospedaje Calle del Sol
568,baker city,US,44.7749,-117.8344,48,Geiser Grand Hotel
573,nizhneangarsk,RU,55.7833,109.5500,55,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

ImportError: Geographic projection support requires: geoviews.